<a href="https://colab.research.google.com/github/SwarajShinde/Awesome-pytorch-list/blob/master/OSIC_kaggle1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import quantreg


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


OSIC CLean Kernal

In [4]:
train = pd.read_csv( '/content/train.csv' )
test  = pd.read_csv( '/content/test.csv' )

train['traintest'] = 0
test ['traintest'] = 1

sub   = pd.read_csv( '/content/sample_submission.csv' )
sub['Weeks']   = sub['Patient_Week'].apply( lambda x: int(x.split('_')[-1]) )
sub['Patient'] = sub['Patient_Week'].apply( lambda x: x.split('_')[0] ) 

print( train.shape, test.shape, sub.shape )

sub.head()

(1549, 8) (5, 8) (730, 5)


,Patient_Week,FVC,Confidence,Weeks,Patient
0,ID00419637202311204720264_-12,2000,100,-12,ID00419637202311204720264
1,ID00421637202311550012437_-12,2000,100,-12,ID00421637202311550012437
2,ID00422637202311677017371_-12,2000,100,-12,ID00422637202311677017371
3,ID00423637202312137826377_-12,2000,100,-12,ID00423637202312137826377
4,ID00426637202313170790466_-12,2000,100,-12,ID00426637202313170790466


In [5]:

train = pd.concat( (train,test) )
train.sort_values( ['Patient','Weeks'], inplace=True )
train.shape

(1554, 8)

In [6]:
train['Sex']           = pd.factorize( train['Sex'] )[0]
train['SmokingStatus'] = pd.factorize( train['SmokingStatus'] )[0]
train

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,traintest
0,ID00007637202177411956430,-4,2315,58.253649,79,0,0,0
1,ID00007637202177411956430,5,2214,55.712129,79,0,0,0
2,ID00007637202177411956430,7,2061,51.862104,79,0,0,0
3,ID00007637202177411956430,9,2144,53.950679,79,0,0,0
4,ID00007637202177411956430,11,2069,52.063412,79,0,0,0
...,...,...,...,...,...,...,...,...
1544,ID00426637202313170790466,13,2712,66.594637,73,0,1,0
1545,ID00426637202313170790466,19,2978,73.126412,73,0,1,0
1546,ID00426637202313170790466,31,2908,71.407524,73,0,1,0
1547,ID00426637202313170790466,43,2975,73.052745,73,0,1,0


In [7]:
train['Percent']       = (train['Percent'] - train['Percent'].mean()) / train['Percent'].std()
train['Age']           = (train['Age'] - train['Age'].mean()) / train['Age'].std()
train['Sex']           = (train['Sex'] - train['Sex'].mean()) / train['Sex'].std()
train['SmokingStatus'] = (train['SmokingStatus'] - train['SmokingStatus'].mean()) / train['SmokingStatus'].std()
train.head(10)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,traintest
0,ID00007637202177411956430,-4,2315,-0.980824,1.672875,-0.514074,-0.653724,0
1,ID00007637202177411956430,5,2214,-1.109228,1.672875,-0.514074,-0.653724,0
2,ID00007637202177411956430,7,2061,-1.303741,1.672875,-0.514074,-0.653724,0
3,ID00007637202177411956430,9,2144,-1.198221,1.672875,-0.514074,-0.653724,0
4,ID00007637202177411956430,11,2069,-1.293571,1.672875,-0.514074,-0.653724,0
5,ID00007637202177411956430,17,2101,-1.252888,1.672875,-0.514074,-0.653724,0
6,ID00007637202177411956430,29,2000,-1.381292,1.672875,-0.514074,-0.653724,0
7,ID00007637202177411956430,41,2064,-1.299927,1.672875,-0.514074,-0.653724,0
8,ID00007637202177411956430,57,2057,-1.308827,1.672875,-0.514074,-0.653724,0
9,ID00009637202177434476278,8,3660,0.384760,0.254782,-0.514074,-0.653724,0


In [8]:
modelL = quantreg('FVC ~ Weeks+Percent+Age+Sex+SmokingStatus', train).fit( q=0.15 )
model  = quantreg('FVC ~ Weeks+Percent+Age+Sex+SmokingStatus', train).fit( q=0.50 )
modelH = quantreg('FVC ~ Weeks+Percent+Age+Sex+SmokingStatus', train).fit( q=0.85 )
print(model.summary())

                         QuantReg Regression Results                          
Dep. Variable:                    FVC   Pseudo R-squared:               0.6079
Model:                       QuantReg   Bandwidth:                       119.9
Method:                 Least Squares   Sparsity:                        649.4
Date:                Tue, 01 Sep 2020   No. Observations:                 1554
Time:                        11:06:53   Df Residuals:                     1548
                                        Df Model:                            5
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      2716.4305     13.996    194.089      0.000    2688.978    2743.883
Weeks            -0.6396      0.356     -1.797      0.073      -1.338       0.059
Percent         644.2268      8.428     76.436      0.000     627.695     660.759
Age            -117.0701      8.331  

In [9]:
train['ypredL'] = modelL.predict( train ).values
train['ypred']  = model.predict( train ).values
train['ypredH'] = modelH.predict( train ).values
train['ypredstd'] = 0.5*np.abs(train['ypredH'] - train['ypred'])+0.5*np.abs(train['ypred'] - train['ypredL'])
train.head(10)

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,traintest,ypredL,ypred,ypredH,ypredstd
0,ID00007637202177411956430,-4,2315,-0.980824,1.672875,-0.514074,-0.653724,0,1887.830905,2112.437791,2306.053836,209.111465
1,ID00007637202177411956430,5,2214,-1.109228,1.672875,-0.514074,-0.653724,0,1803.975430,2023.959698,2209.660194,202.842382
2,ID00007637202177411956430,7,2061,-1.303741,1.672875,-0.514074,-0.653724,0,1687.281937,1897.369976,2062.892501,187.805282
3,ID00007637202177411956430,9,2144,-1.198221,1.672875,-0.514074,-0.653724,0,1747.845631,1964.069538,2142.709303,197.431836
4,ID00007637202177411956430,11,2069,-1.293571,1.672875,-0.514074,-0.653724,0,1689.737141,1901.363563,2070.829705,190.546282
5,ID00007637202177411956430,17,2101,-1.252888,1.672875,-0.514074,-0.653724,0,1708.441741,1923.734391,2101.937588,196.747924
6,ID00007637202177411956430,29,2000,-1.381292,1.672875,-0.514074,-0.653724,0,1621.921129,1833.337354,2005.736228,191.907549
7,ID00007637202177411956430,41,2064,-1.299927,1.672875,-0.514074,-0.653724,0,1659.330330,1878.079010,2067.951993,204.310831
8,ID00007637202177411956430,57,2057,-1.308827,1.672875,-0.514074,-0.653724,0,1639.858642,1862.111485,2062.256764,211.199061
9,ID00009637202177434476278,8,3660,0.384760,0.254782,-0.514074,-0.653724,0,2879.168686,3150.524311,3541.024439,330.927876


In [10]:
def metric( trueFVC, predFVC, predSTD ):
    
    clipSTD = np.clip( predSTD, 70 , 9e9 )  
    
    deltaFVC = np.clip( np.abs(trueFVC-predFVC), 0 , 1000 )  

    return np.mean( -1*(np.sqrt(2)*deltaFVC/clipSTD) - np.log( np.sqrt(2)*clipSTD ) )
    

print( 'Metric:', metric( train['FVC'].values, train['ypred'].values, train['ypredstd'].values  ) )

Metric: -7.188729015782974


In [11]:
dt = train.loc[ train.traintest==1 ,['Patient','Percent','Age','Sex','SmokingStatus']]
test = pd.merge( sub, dt, on='Patient', how='left' )
test.sort_values( ['Patient','Weeks'], inplace=True )
test.head(10)

,Patient_Week,FVC,Confidence,Weeks,Patient,Percent,Age,Sex,SmokingStatus
0,ID00419637202311204720264_-12,2000,100,-12,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
5,ID00419637202311204720264_-11,2000,100,-11,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
10,ID00419637202311204720264_-10,2000,100,-10,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
15,ID00419637202311204720264_-9,2000,100,-9,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
20,ID00419637202311204720264_-8,2000,100,-8,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
25,ID00419637202311204720264_-7,2000,100,-7,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
30,ID00419637202311204720264_-6,2000,100,-6,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
35,ID00419637202311204720264_-5,2000,100,-5,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
40,ID00419637202311204720264_-4,2000,100,-4,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724
45,ID00419637202311204720264_-3,2000,100,-3,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724


In [18]:
test['ypred_low'] = modelL.predict(test).values
test['FVC'] = model.predict(test).values
test['ypred_high'] = modelH.predict(test).values

In [19]:
test['Confidence'] = np.abs(test['ypred_high']-test['ypred_low'])/2

In [22]:
test.head()

,Patient_Week,FVC,Confidence,Weeks,Patient,Percent,Age,Sex,SmokingStatus,ypred_low,ypred_high
0,ID00419637202311204720264_-12,2605.566153,257.169229,-12,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724,2368.256891,2882.595349
5,ID00419637202311204720264_-11,2604.926505,257.645465,-11,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724,2367.368512,2882.659443
10,ID00419637202311204720264_-10,2604.286857,258.121702,-10,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724,2366.480134,2882.723537
15,ID00419637202311204720264_-9,2603.647209,258.597938,-9,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724,2365.591755,2882.787630
20,ID00419637202311204720264_-8,2603.007560,259.074174,-8,ID00419637202311204720264,-0.377929,0.822019,-0.514074,-0.653724,2364.703376,2882.851724


In [21]:
test.drop("SVC",axis=1,inplace=True)

In [24]:
test[['Patient_Week','FVC','Confidence']].to_csv('submission1.csv',index=False)


In [25]:
test[['Patient_Week','FVC','Confidence']].head()

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2605.566153,257.169229
5,ID00419637202311204720264_-11,2604.926505,257.645465
10,ID00419637202311204720264_-10,2604.286857,258.121702
15,ID00419637202311204720264_-9,2603.647209,258.597938
20,ID00419637202311204720264_-8,2603.007560,259.074174


In [27]:
ans = pd.read_csv('/content/submission1.csv')

In [28]:
ans

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2605.566153,257.169229
1,ID00419637202311204720264_-11,2604.926505,257.645465
2,ID00419637202311204720264_-10,2604.286857,258.121702
3,ID00419637202311204720264_-9,2603.647209,258.597938
4,ID00419637202311204720264_-8,2603.007560,259.074174
...,...,...,...
725,ID00426637202313170790466_129,2553.779006,326.697755
726,ID00426637202313170790466_130,2553.139358,327.173991
727,ID00426637202313170790466_131,2552.499710,327.650228
728,ID00426637202313170790466_132,2551.860062,328.126464
